In [ ]:
# | default_exp test_client

In [ ]:
# | export

from collections import namedtuple
from unittest.mock import AsyncMock, MagicMock
from typing import *
import inspect
import functools
from datetime import datetime, timedelta
import asyncio

from fastcore.foundation import patch
from fastcore.meta import delegates

from fastkafka.application import FastKafka

In [ ]:
import pytest

from pydantic import BaseModel, Field

from fastkafka.helpers import create_missing_topics
from fastkafka.testing import LocalKafkaBroker

In [ ]:
# | export

class AwaitedMock():
    @staticmethod
    def _await_for(f: Callable[[...], Any]) -> Callable[[...], Any]:
        @delegates(f)
        async def inner(*args, f=f, timeout: int=60, **kwargs)->Any:
            if inspect.iscoroutinefunction(f):
                return await asyncio.wait_for(f(*args, **kwargs), timeout=timeout)
            else:
                t0=datetime.now()
                e: Optional[Exception] = None
                while (datetime.now()-t0 < timedelta(seconds=timeout)):
                    try:
                        return f(*args, **kwargs)
                    except Exception as _e:
                        await asyncio.sleep(1)
                        e = _e
                raise e
        return inner

    def __init__(self, o: Any):
        self._o = o
        
        for name in o.__dir__():
            if not name.startswith("_"):
                f = getattr(o, name)
                if inspect.ismethod(f):
                    setattr(self, name, self._await_for(f))

In [ ]:
# | export


@patch
def create_mocks(self: FastKafka) -> None:
    """Creates self.mocks as a named tuple mapping a new function obtained by calling the original functions and a mock"""
    app_methods = [f for f, _ in self._consumers_store.values()] + [
        f for f, _, _ in self._producers_store.values()
    ]
    self.AppMocks = namedtuple(
        f"{self.__class__.__name__}Mocks", [f.__name__ for f in app_methods]
    )
    
    self.mocks = self.AppMocks(
        **{
            f.__name__: AsyncMock()
            if inspect.iscoroutinefunction(f)
            else MagicMock()
            for f in app_methods
        }
    )
    
    self.awaited_mocks = self.AppMocks(
        **{name: AwaitedMock(mock)
        for name, mock in self.mocks._asdict().items()}
    )

    def add_mock(f: Callable[[...], Any], mock: Union[AsyncMock, MagicMock]) -> Callable[[...], Any]:
        """Add call to mock when calling function f"""
        @functools.wraps(f)
        async def async_inner(
            *args, f: Callable[[...], Any] = f, mock: AsyncMock = mock, **kwargs
        ) -> Any:
            await mock(*args, **kwargs)
            return await f(*args, **kwargs)

        @functools.wraps(f)
        def sync_inner(
            *args, f: Callable[[...], Any] = f, mock: MagicMock = mock, **kwargs
        ) -> Any:
            mock(*args, **kwargs)
            return f(*args, **kwargs)

        if inspect.iscoroutinefunction(f):
            return async_inner
        else:
            return sync_inner

    self._consumers_store = {
        name: (
            add_mock(f, getattr(self.mocks, f.__name__)),
            kwargs,
        )
        for name, (f, kwargs) in self._consumers_store.items()
    }
    self._producers_store = {
        name: (
            add_mock(f, getattr(self.mocks, f.__name__)),
            producer,
            kwargs,
        )
        for name, (f, producer, kwargs) in self._producers_store.items()
    }

In [ ]:
class TestMsg(BaseModel):
    msg: str = Field(...)


def create_app(bootstrap_servers: str) -> FastKafka:
    app = FastKafka(bootstrap_servers=bootstrap_servers)

    @app.consumes()
    async def on_preprocessed_signals(msg: TestMsg):
        await to_predictions(TestMsg(msg="prediction"))

    @app.produces()
    async def to_predictions(prediction: TestMsg) -> TestMsg:
        print(f"Sending prediction: {prediction}")
        return prediction

    return app

In [ ]:
app = create_app("localhost:9092")

app.create_mocks()
app.mocks.on_preprocessed_signals.assert_not_awaited()
app.mocks.to_predictions.assert_not_awaited()
app.create_mocks()
app.mocks.on_preprocessed_signals.assert_not_awaited()
app.mocks.to_predictions.assert_not_awaited()

In [ ]:
with pytest.raises(AssertionError) as e:
    await app.awaited_mocks.on_preprocessed_signals.assert_called_with(123, timeout=2)

In [ ]:
app = create_app("localhost:9092")

app.create_mocks()
app.mocks.on_preprocessed_signals.assert_not_awaited()
await app.awaited_mocks.on_preprocessed_signals.assert_not_awaited(timeout=3)

In [ ]:
# | export


class Tester(FastKafka):
    def __init__(self, app: Union[FastKafka, List[FastKafka]]):
        self.apps = app if isinstance(app, list) else [app]
        super().__init__(
            bootstrap_servers=self.apps[0]._kafka_config["bootstrap_servers"]
        )
        self.create_mirrors()

    async def startup(self):
        for app in self.apps:
            app.create_mocks()
            await app.startup()

        self.create_mocks()
        await super().startup()
        await asyncio.sleep(3)

    async def shutdown(self):
        await super().shutdown()
        for app in self.apps[::-1]:
            await app.shutdown()

    def create_mirrors(self):
        pass

In [ ]:
tester = Tester([create_app("localhost:9092")])

with pytest.raises(AttributeError) as e:
    tester.mocks

In [ ]:
async with LocalKafkaBroker(
    zookeeper_port=9998, listener_port=9789
) as bootstrap_server:
    create_missing_topics(
        ["preprocessed_signals", "predictions"],
        bootstrap_servers=bootstrap_server,
        num_partitions=1,
    )
    tester = Tester(create_app(bootstrap_servers=bootstrap_server))

    @tester.produces()
    async def to_preprocessed_signals(msg: TestMsg) -> TestMsg:
        print(f"Producing msg {msg}")
        return msg

    tester.to_preprocessed_signals = to_preprocessed_signals

    @tester.consumes(auto_offset_reset="latest")
    async def on_predictions(msg: TestMsg):
        pass

    async with tester:
        await tester.to_preprocessed_signals(TestMsg(msg="signal"))
        await asyncio.sleep(5)
        tester.mocks.on_predictions.assert_called()

print("ok")

[INFO] fastkafka.testing: Java is already installed.
[INFO] fastkafka.testing: But not exported to PATH, exporting...
[INFO] fastkafka.testing: Kafka is already installed.
[INFO] fastkafka.testing: But not exported to PATH, exporting...
[INFO] fastkafka.testing: Starting zookeeper...
[INFO] fastkafka.testing: Zookeeper started, sleeping for 5 seconds...
[INFO] fastkafka.testing: Starting Kafka broker...
[INFO] fastkafka.testing: Kafka broker started, sleeping for 5 seconds...
[INFO] fastkafka.testing: Local Kafka broker up and running on 127.0.0.1:9789
[INFO] fastkafka.helpers: create_missing_topics(['preprocessed_signals', 'predictions']): new_topics = [NewTopic(topic=preprocessed_signals,num_partitions=1), NewTopic(topic=predictions,num_partitions=1)]
[INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:9789'}'
[INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers':

In [ ]:
# | export


def mirror_producer(
    topic: str, producer_f: Callable[[...], Any]
) -> Callable[[...], Any]:
    msg_type = inspect.signature(producer_f).return_annotation

    async def skeleton_func(msg):
        pass

    mirror_func = skeleton_func
    sig = inspect.signature(skeleton_func)

    # adjust name
    mirror_func.__name__ = "on_" + topic

    # adjust arg and return val
    sig = sig.replace(
        parameters=[
            inspect.Parameter(
                name="msg",
                annotation=msg_type,
                kind=inspect.Parameter.POSITIONAL_OR_KEYWORD,
            )
        ]
    )

    mirror_func.__signature__ = sig

    return mirror_func

In [ ]:
app = FastKafka()


@app.produces()
def to_topic1() -> TestMsg:
    pass


@app.produces(topic="topic2")
def some_log(in_var: int) -> TestMsg:
    pass


for topic, (producer_f, _, _) in app._producers_store.items():
    mirror = mirror_producer(topic, producer_f)
    assert mirror.__name__ == "on_" + topic
    assert inspect.signature(mirror).parameters["msg"] == inspect.Parameter(
        name="msg",
        annotation=TestMsg,
        kind=inspect.Parameter.POSITIONAL_OR_KEYWORD,
    )

In [ ]:
# | export


def mirror_consumer(
    topic: str, consumer_f: Callable[[...], Any]
) -> Callable[[...], Any]:
    msg_type = inspect.signature(consumer_f).parameters["msg"]

    async def skeleton_func(msg):
        return msg

    mirror_func = skeleton_func
    sig = inspect.signature(skeleton_func)

    # adjust name
    mirror_func.__name__ = "to_" + topic

    # adjust arg and return val
    sig = sig.replace(parameters=[msg_type], return_annotation=msg_type.annotation)

    mirror_func.__signature__ = sig
    return mirror_func

In [ ]:
app = FastKafka()


@app.consumes()
def on_topic1(msg: TestMsg):
    pass


@app.consumes(topic="topic2")
def some_log(msg: TestMsg):
    pass


for topic, (consumer_f, _) in app._consumers_store.items():
    mirror = mirror_consumer(topic, consumer_f)
    assert mirror.__name__ == "to_" + topic
    assert inspect.signature(mirror).return_annotation == TestMsg, inspect.signature(
        mirror
    ).return_annotation
    assert inspect.signature(mirror).parameters["msg"] == inspect.Parameter(
        name="msg",
        annotation=TestMsg,
        kind=inspect.Parameter.POSITIONAL_OR_KEYWORD,
    )

In [ ]:
# | export

@patch
def create_mirrors(self: Tester):
    for app in self.apps:
        for topic, (consumer_f, _) in app._consumers_store.items():
            mirror_f = mirror_consumer(topic, consumer_f)
            mirror_f = self.produces()(mirror_f)
            setattr(self, mirror_f.__name__, mirror_f)
        for topic, (producer_f, _, _) in app._producers_store.items():
            mirror_f = mirror_producer(topic, producer_f)
            mirror_f = self.consumes()(mirror_f)
            setattr(self, mirror_f.__name__, mirror_f)     

In [ ]:
async with LocalKafkaBroker(zookeeper_port=9998, listener_port=9789) as bootstrap_server:
    create_missing_topics(
        ["preprocessed_signals", "predictions"],
        bootstrap_servers=bootstrap_server,
        num_partitions=1,
    )
    
    tester = Tester(create_app(bootstrap_servers=bootstrap_server))
    
    async with tester:
        await tester.to_preprocessed_signals(TestMsg(msg="signal"))
        await tester.awaited_mocks.on_predictions.assert_called(timeout=5)
print("ok")

[INFO] fastkafka.testing: Java is already installed.
[INFO] fastkafka.testing: Kafka is already installed.
[INFO] fastkafka.testing: Starting zookeeper...
[INFO] fastkafka.testing: Zookeeper started, sleeping for 5 seconds...
[INFO] fastkafka.testing: Starting Kafka broker...
[INFO] fastkafka.testing: Kafka broker started, sleeping for 5 seconds...
[INFO] fastkafka.testing: Local Kafka broker up and running on 127.0.0.1:9789
[INFO] fastkafka.helpers: create_missing_topics(['preprocessed_signals', 'predictions']): new_topics = [NewTopic(topic=preprocessed_signals,num_partitions=1), NewTopic(topic=predictions,num_partitions=1)]
[INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:9789'}'
[INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:9789'}'
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._co